In [1]:
import re
import quandl
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from matplotlib import style
style.use('fivethirtyeight')
api_key = open('quandlapikey.txt', 'r').read()

In [23]:

#get 50 states using pandas read html from wikipedia

def state_list():
    fifty_states = pd.read_html('https://en.wikipedia.org/wiki/List_of_states_and_territories_of_the_United_States')
    return fifty_states[0].iloc[:, 1]


def grab_initial_state_data():
    
    states = state_list()
    
    main_df = pd.DataFrame()
        
    df = quandl.get('FMAC/HPI', authtoken=api_key)
    for abbv in states:
        df[abbv] = (df[abbv]-df[abbv][0]) / df[abbv][0] * 100.0 # change to % info and look graph1
        #main_df['NSA Value' + abbv] = df['NSA Value']
        #main_df['SA Value' + abbv] = df['SA Value']

        #print(query)
        if main_df.empty:
            main_df = df
        else:
            main_df = main_df.join(df, lsuffix=abbv)
            
    pickle_out = open('fifty_states4.pickle', 'wb')
    pickle.dump(main_df, pickle_out)
    pickle_out.close()

    
def HPI_Benchmark():
    df = quandl.get("FMAC/HPI_USA", authtoken=api_key)
    df["NSA Value"] = (df['NSA Value'] - df['NSA Value'][0]) / df['NSA Value'][0] * 100.0
    return df
    
def mortgage_30y():
    df = quandl.get("FMAC/MORTG", trim_start='1975-01-01', authtoken=api_key)
    df["Value"] = (df['Value'] - df['Value'][0]) / df['Value'][0] * 100.0
    df.rename(columns={'Value':'M30'}, inplace=True)
    df = df.resample('M').mean()
    df.columns = ['M30']      
    return df



In [24]:
HPI_data = pd.read_pickle('fiddy_states3.pickle')
m30 = mortgage_30y()
HPI_Bench = HPI_Benchmark()
m30.columns=['M30']
HPI = HPI_Bench.join(m30)
print(HPI.head())

            NSA Value   SA Value       M30
Date                                      
1975-01-31   0.000000  23.587964  0.000000
1975-02-28   0.620939  23.738887 -3.393425
1975-03-31   1.666642  23.954488 -5.620361
1975-04-30   3.034074  24.224848 -6.468717
1975-05-31   3.851987  24.367024 -5.514316


In [25]:
print(HPI.corr())

           NSA Value  SA Value       M30
NSA Value   1.000000  0.999811 -0.777935
SA Value    0.999811  1.000000 -0.778349
M30        -0.777935 -0.778349  1.000000


In [26]:
state_HPI_M30 = HPI_data.join(m30)
print(state_HPI_M30.corr())

             NSA ValueAL  SA ValueAL  NSA ValueAK  SA ValueAK  NSA ValueAZ  \
NSA ValueAL     1.000000    0.999791     0.955979    0.955735     0.946364   
SA ValueAL      0.999791    1.000000     0.955789    0.955938     0.946330   
NSA ValueAK     0.955979    0.955789     1.000000    0.999780     0.925931   
SA ValueAK      0.955735    0.955938     0.999780    1.000000     0.925876   
NSA ValueAZ     0.946364    0.946330     0.925931    0.925876     1.000000   
SA ValueAZ      0.946209    0.946397     0.925769    0.925942     0.999929   
NSA ValueAR     0.995937    0.995819     0.973941    0.973783     0.946409   
SA ValueAR      0.995768    0.995959     0.973772    0.973918     0.946380   
NSA ValueCA     0.948956    0.948863     0.934404    0.934300     0.982048   
SA ValueCA      0.948791    0.948980     0.934244    0.934418     0.981987   
NSA ValueCO     0.951212    0.951030     0.951306    0.951138     0.933707   
SA ValueCO      0.951045    0.951260     0.951171    0.951378   

In [27]:
print(state_HPI_M30.corr()['M30'])

NSA ValueAL   -0.795999
SA ValueAL    -0.796358
NSA ValueAK   -0.734597
SA ValueAK    -0.735067
NSA ValueAZ   -0.673148
SA ValueAZ    -0.673366
NSA ValueAR   -0.788459
SA ValueAR    -0.788758
NSA ValueCA   -0.732881
SA ValueCA    -0.733168
NSA ValueCO   -0.787205
SA ValueCO    -0.787771
NSA ValueCT   -0.750014
SA ValueCT    -0.750511
NSA ValueDE   -0.791559
SA ValueDE    -0.791742
NSA ValueFL   -0.681983
SA ValueFL    -0.682191
NSA ValueGA   -0.754488
SA ValueGA    -0.754952
NSA ValueHI   -0.794900
SA ValueHI    -0.794985
NSA ValueID   -0.767052
SA ValueID    -0.767541
NSA ValueIL   -0.780371
SA ValueIL    -0.780928
NSA ValueIN   -0.806687
SA ValueIN    -0.807473
NSA ValueIA   -0.820573
SA ValueIA    -0.821136
                 ...   
SA ValueOK    -0.690278
NSA ValueOR   -0.790972
SA ValueOR    -0.791476
NSA ValuePA   -0.813025
SA ValuePA    -0.813560
NSA ValueRI   -0.763211
SA ValueRI    -0.763475
NSA ValueSC   -0.791170
SA ValueSC    -0.791448
NSA ValueSD   -0.826046
SA ValueSD    -0

In [28]:
print(state_HPI_M30.corr()['M30'].describe())

count    101.000000
mean      -0.756050
std        0.181625
min       -0.826404
25%       -0.800348
50%       -0.787771
75%       -0.761960
max        1.000000
Name: M30, dtype: float64
